In [ ]:
import matplotlib.pyplot as plt
from pya import *

%matplotlib widget

# Simulation

In [ ]:
import simulation

n = 128
sim_speed = 0.004
sim_fps = 400
duration = 6
frame_amount = duration * sim_fps

initial_state = np.array([[simulation.gaussian_impulse(x, y, n, offset=[-0.6, 0.0], width=0.15, impulse=-0.1) for x in range(n)] for y in range(n)])
# initial_state = np.array([[simulation.gaussian(x, y, n, offset=[-0.6, 0.0], width=0.05) for x in range(n)] for y in range(n)]) + np.array([[simulation.gaussian(x, y, n, offset=[-0.62, 0.0], width=0.05, imag=True) for x in range(n)] for y in range(n)])
# potential = np.array([[simulation.parabola(x, y, n, offset=[0, 0], factor=10000) for x in range(n)] for y in range(n)])
potential = np.array([[complex(0, 0) for x in range(n)] for y in range(n)])

multi_slit = [(-15, -13), (-8, -6), (-1, 1), (6, 8), (13, 15)]
double_slit = [(-4, -2), (2, 4)]
single_slit = [(-2, 2)]
slits = double_slit

In [ ]:
plt.pcolormesh(np.abs(initial_state), cmap='inferno', vmin=0, vmax=1)
# plt.pcolormesh(np.abs(potential), vmin=0, vmax=20000)
plt.colorbar()
plt.show()

In [ ]:
# action
frames = simulation.sim(n, sim_fps, duration, slits, sim_speed, initial_state=initial_state, potential=potential)

# Video

In [ ]:
import video

# save video
video_filename, anim = video.create(frames, 20, frame_amount, sim_fps, slits, n)

# plt.close()